In [1]:
import pandas as pd
import numpy as np

In [2]:
example_data = pd.read_excel('./Example_Data.xlsx', sheet_name='Example_Data', header=[0,1])
example_db = pd.read_excel('./Example_Data.xlsx', sheet_name='Example_DB', header=0)
example_answer = pd.read_excel('./Example_Data.xlsx', sheet_name='Example_Answer', header=0)
#extract headers
header_level1 = example_data.columns.levels[0]
company_info = example_data[header_level1[0]].columns
company_metric = example_data[header_level1[1]].columns
example_data = example_data.droplevel(level=0, axis=1)

# Task 1: Data Cleaning

In [17]:
#Company ID and Company Name should be corresponding 
#if there is one ID corresponding to multiple Names or one Name corresponding to multiple IDs, 
#the ID/Name is invalid and needs to be cleaned up.
index_remove = pd.Index([])

# remove the row where the id corresponds to multiple names
example_data_group = example_data.groupby(['Company ID', 'Company Name'])
data_indices = example_data_group.indices
data_groups = example_data_group.groups
data_dict = {}
for c_id1,c_name1 in data_indices:
    if c_id1 not in data_dict:
        data_dict[c_id1] = c_name1
    else:
        old_name = data_dict[c_id1]
        new_name = c_name1
        if len(data_groups[(c_id1, old_name)])<len(data_groups[(c_id1, new_name)]):
            data_dict[c_id1] = new_name

pair_list = [(key, values) for key, values in data_dict.items()]
pair_remove = [item for item in list(data_indices) if item not in pair_list]

for pair in pair_remove:
    index_remove = index_remove.union(data_groups[pair])

In [18]:
# remove the row where the name corresponds to multiple id
example_data_group = example_data.groupby(['Company Name', 'Company ID'])
data_indices = example_data_group.indices
data_groups = example_data_group.groups
data_dict = {}
for c_id1,c_name1 in data_indices:

    if c_id1 not in data_dict:
        data_dict[c_id1] = c_name1
    else:
        old_name = data_dict[c_id1]
        new_name = c_name1
        if len(data_groups[(c_id1, old_name)])<len(data_groups[(c_id1, new_name)]):
            data_dict[c_id1] = new_name

pair_list = [(key, values) for key, values in data_dict.items()]
pair_remove =[item for item in list(data_indices) if item not in pair_list]
for pair in pair_remove:
    index_remove = index_remove.union(data_groups[pair])

example_data = example_data.drop(labels = index_remove, axis=0)

In [19]:
#Fiscal Year is the year, between 1999 and 2021.
df = example_data
df = df[df['Fiscal Year'].apply(lambda x: str(x).isdigit())] #remove non digit data
df = df[(df['Fiscal Year']>1999) & (df['Fiscal Year']<2021)] #remove data outside the range

In [20]:
#SIC Code is a four-digit number.
df = df[df['SIC Code'].apply(lambda x: str(x).isdigit() and len(str(x)) == 4)]

In [21]:
#Trading Currency can only be one of two units, USD or GBP.
df = df[(df['Trading Currency'] == 'USD') | (df['Trading Currency'] == 'GBP')] 

In [22]:
#SP / CDS / APD / ARD / ADA is an integer number.
for key in company_metric:
    df = df[df[key].apply(lambda x: int(x)==x if pd.notnull(x) else True)]  #remove data that is not intrger number
    df[key] = df[key].apply(lambda x:int(x) if pd.notnull(x) else np.nan)  #conver float data to integer
    #df[company_metric] = df[company_metric].astype('Int64')

# Task 2: Data Processing

In [9]:
headers = example_db.columns
data_db = pd.DataFrame(columns=headers)

In [10]:
company_info_df = df[company_info]

for key in company_metric:
    metric_values = pd.DataFrame({'Metric Name': [key]*len(df),
                                  'Value': df[company_metric][key]})
    batch_data = pd.concat([company_info_df,metric_values], axis =1)
    data_db = pd.concat([data_db, batch_data], axis = 0, ignore_index=True)

# Task 3: Data Comparison

In [11]:
# 1. Unequal 2. Not in DB 3. Not in File
answer_headers = ['Company Info']*7 + ['Data in DB','Data in File','ERROR Type']
data_answer = pd.DataFrame(columns=answer_headers)
data_answer_list = []

In [12]:
tmp_db = example_db.copy()
for x in data_db.index:
    row_db = data_db.loc[x,headers]
    row_company_info = row_db[:-1]
    row_value = row_db[-1]
    
    row_target_db = tmp_db[(tmp_db[headers[:-1]] == row_company_info).all(1)]
    
    # error not in db
    if not len(row_target_db):
        error_type = 'Not_in_DB'
        data_in_db = np.nan
        data_in_file = row_value
        data_answer_list.append(list(row_company_info)+[data_in_db, data_in_file, error_type])
    
    else:
        row_db_value = row_target_db['Value'].item()
        
        # error not equal
        if not ((row_db_value == row_value) | (pd.isnull(row_value) & pd.isnull(row_db_value))):
            error_type = 'UnEqual'
            data_in_db = row_target_db['Value'].item()
            data_in_file = row_value
            data_answer_list.append(list(row_company_info)+[data_in_db, data_in_file, error_type])
        target_idx = row_target_db.index
        tmp_db = tmp_db.drop(labels = target_idx, axis = 0)
            
data_answer = pd.DataFrame(data=data_answer_list, columns=answer_headers)
data_answer_not_in_file = pd.DataFrame(columns=answer_headers)

# error not in file
if len(tmp_db):
    data_answer_not_in_file['ERROR Type'] = ['Not_in_File']*len(tmp_db)
    data_answer_not_in_file['Data in DB'] = list(tmp_db['Value'])
    data_answer_not_in_file['Company Info'] = tmp_db[headers[:-1]].values
data_answer = pd.concat([data_answer,data_answer_not_in_file], axis = 0, ignore_index=True)

In [13]:
data_answer.loc[data_answer['ERROR Type'] == 'UnEqual']

,Company Info,Company Info,Company Info,Company Info,Company Info,Company Info,Company Info,Data in DB,Data in File,ERROR Type
17,4986826,L Company,2015,Auto Components,6090,USD,SP,17.0,179.0,UnEqual
18,4986826,L Company,2017,Auto Components,6090,USD,SP,19.0,196.0,UnEqual


In [14]:
# There are many rows with 'Not_in_File' error, this is due to the frist step of data cleanning, we removed many data
# that corresponds to multiple IDs and names in 'example_data', but they still exist in 'example_db', so we won't be 
# able to find any records in 'example_data'
data_answer

,Company Info,Company Info,Company Info,Company Info,Company Info,Company Info,Company Info,Data in DB,Data in File,ERROR Type
0,4991368,B Company,2010,Auto Components,4937,USD,SP,NaN,18.0,Not_in_DB
1,4991368,B Company,2011,Auto Components,4937,USD,SP,NaN,22.0,Not_in_DB
2,4991368,B Company,2012,Auto Components,4937,USD,SP,NaN,21.0,Not_in_DB
3,4991368,B Company,2013,Auto Components,4937,USD,SP,NaN,37.0,Not_in_DB
4,4991368,B Company,2014,Auto Components,2937,USD,SP,NaN,44.0,Not_in_DB
...,...,...,...,...,...,...,...,...,...,...
133,4994275,D Company,2015,Machinery,5722,USD,ADA,0.0,NaN,Not_in_File
134,4994275,D Company,2016,Machinery,5722,USD,ADA,0.0,NaN,Not_in_File
135,4994275,D Company,2017,Machinery,5722,USD,ADA,0.0,NaN,Not_in_File
136,4994275,D Company,2018,Machinery,5722,USD,ADA,0.0,NaN,Not_in_File
